In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from keras import models
from keras import layers

In [2]:
(boston_train_data, boston_train_targets), (boston_test_data, boston_test_targets) = tf.keras.datasets.boston_housing.load_data(
    path='boston_housing.npz', test_split=0.2, seed=113)

In [3]:
scaled_train_data = preprocessing.scale(boston_train_data)
scaled_test_data = preprocessing.scale(boston_test_data)

In [4]:
input_size = 13
output_size = 1
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation ='relu')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['mae'])


2022-01-18 02:36:39.845144: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-18 02:36:39.846757: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
kf = 5

num_val_samples = len(boston_train_data) // kf
num_epochs = 100
all_scores = []

for i in range(kf):
    
    print('processing fold #', i)
    val_data = scaled_train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = boston_train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
    [scaled_train_data[:i * num_val_samples],
    scaled_train_data[(i + 1) * num_val_samples:]],
    axis=0)
    partial_train_targets = np.concatenate(
    [boston_train_targets[:i * num_val_samples],
    boston_train_targets[(i + 1) * num_val_samples:]],
    axis=0)



    model.fit(partial_train_data, partial_train_targets,
    epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4


In [7]:
all_scores

[1.1221199035644531,
 1.164147973060608,
 0.9680194854736328,
 0.8719687461853027,
 0.7279623746871948]

In [8]:
np.mean(all_scores)

0.9708436965942383

In [9]:

model.fit(scaled_train_data, boston_train_targets,
epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(scaled_test_data, boston_test_targets)

4/4 [==============================] - 0s 10ms/step - loss: 14.5754 - mae: 2.6029
